In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import re
from sklearn.preprocessing import MinMaxScaler
from keras .models import Sequential
from keras.layers import Dense,LSTM,Dropout,Flatten
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
path,data,x1 = list(),list(),list()
s=0
datefile = os.walk(r'E:\资料\Dataset\PHM 2010\PHM 2010\c1\c1')
datay =pd.read_csv(r'E:\资料\Dataset\PHM 2010\PHM 2010\c1\c1_wear.csv')
datay = datay.iloc[:,1:2]
for dirs,folders,fname in datefile:
    for name in fname:
        path.append(os.path.join(dirs,name))
for i in path:
    s+=1
    data1 = pd.read_csv(i,names=['fx','fy','fz','ax','ay','az','h'])
    x = re.match(r'.*(c_\d_\d{3}).csv$',i).group(1)
    data2 = pd.DataFrame()
    for d in range(1500):
        data2 = data2.append(pd.DataFrame(data1.iloc[d*80:d*80+2]),ignore_index=True)
    print(s)   
    data.append(data2)
    x1.append(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [4]:
dataset_x =pd.concat(data,keys=x1)
dataset_x

fx      fy      fz     ax     ay     az      h
c_1_001 0      0.704  -0.387  -1.084  0.018  0.031  0.027 -0.004
        1      0.772  -0.573  -1.153 -0.056 -0.057 -0.058 -0.004
        2     -0.693   1.297   0.770  0.015 -0.002 -0.012 -0.004
        3     -0.665   1.234   0.803 -0.031 -0.047 -0.045 -0.004
        4      1.137  -0.305   0.565 -0.018 -0.019 -0.011 -0.004
...              ...     ...     ...    ...    ...    ...    ...
c_1_315 2995  93.360  37.399  52.300  0.048 -0.030 -0.053  0.228
        2996 -15.678  -1.370  22.169  0.117  0.143  0.056  0.233
        2997 -13.496  -1.542  23.558  0.149 -0.110 -0.268  0.233
        2998  46.602   9.765 -14.901 -0.301 -0.007 -0.163  0.234
        2999  41.871  10.031 -11.355 -0.153  0.321  0.219  0.234

[945000 rows x 7 columns]

In [5]:
# sc=MinMaxScaler(feature_range=[0,1])
# train_x_set = sc.fit_transform(dataset_x)
# train_y_set = sc.fit_transform(datay)
train_x_set = np.array(dataset_x)
train_y_set = np.array(datay)
train_x_set = np.reshape(train_x_set,(315,3000,7))

In [6]:
# train_x_set=np.reshape(train_x_set,(315,3000,7))

In [7]:
train_x_set

array([[[ 7.0400e-01, -3.8700e-01, -1.0840e+00, ...,  3.1000e-02,
          2.7000e-02, -4.0000e-03],
        [ 7.7200e-01, -5.7300e-01, -1.1530e+00, ..., -5.7000e-02,
         -5.8000e-02, -4.0000e-03],
        [-6.9300e-01,  1.2970e+00,  7.7000e-01, ..., -2.0000e-03,
         -1.2000e-02, -4.0000e-03],
        ...,
        [-3.5100e-01,  1.5690e+00,  7.1300e-01, ..., -9.0000e-02,
         -8.1000e-02, -1.0000e-03],
        [ 1.1790e+00, -3.2600e-01,  2.5930e+00, ...,  1.1600e-01,
          1.1400e-01, -2.0000e-03],
        [ 1.1930e+00, -3.3200e-01,  2.7040e+00, ..., -3.9000e-02,
         -3.0000e-02, -2.0000e-03]],

       [[-1.0000e-01, -1.3970e+00,  1.5410e+00, ..., -9.7000e-02,
         -9.7000e-02, -5.0000e-03],
        [-1.7400e-01, -1.4720e+00,  1.4890e+00, ..., -4.6000e-02,
         -5.1000e-02, -5.0000e-03],
        [-2.6400e-01, -1.3150e+00,  1.1270e+00, ..., -3.9000e-02,
         -3.3000e-02, -5.0000e-03],
        ...,
        [ 2.8000e+00,  2.0790e+00,  7.9000e-02, ..., -

In [8]:
x_train= train_x_set[:225,:]

In [9]:
y_train=train_y_set[:225,:]

In [11]:
from keras.layers import Conv1D, MaxPooling1D

In [12]:
model = Sequential()
#LSTM 第一层
model.add(LSTM(32,return_sequences=True,input_shape=(x_train.shape[1], 7)))
model.add(Dropout(0.3))
model.add(Conv1D(filters=80, kernel_size=3, padding='same', activation='relu', kernel_initializer="glorot_uniform"))
model.add(MaxPooling1D(pool_size=2, padding='valid'))
model.add(Conv1D(filters=54, kernel_size=3, padding='same', activation='relu', kernel_initializer="glorot_uniform"))
model.add(Conv1D(filters=54, kernel_size=3, padding='same', activation='relu', kernel_initializer="glorot_uniform"))
    # model.add(MaxPooling1D(pool_size=2, padding='valid'))
    # model.add(LSTM(32,return_sequences=True))
    # model.add(LSTM(32,return_sequences=False))
    # model.add(Dropout(0.2))
    # model.add(Dense(32, activation="relu", kernel_initializer="uniform"))
model.add(Flatten())
model.add(Dropout(0.3))
# #第三层
# model.add(LSTM(100, return_sequences=False))
# model.add(Dropout(0.2))
# , activation="relu", kernel_initializer="uniform"
# Dense层
model.add(Dense(1, activation="relu", kernel_initializer="uniform"))

In [13]:
# 模型编译
model.compile(optimizer=Adam(0.0001),loss='mse', metrics=['mae'])
# ,validation_split=0.2,verbose=2
# 训练模型
model.fit(x_train, y_train, epochs=40, batch_size=50,validation_split=0.2,verbose=2)

Train on 180 samples, validate on 45 samples
Epoch 1/40
 - 8s - loss: 8216.7403 - mae: 89.8888 - val_loss: 12671.5498 - val_mae: 112.4641
Epoch 2/40
 - 6s - loss: 8135.8033 - mae: 89.4394 - val_loss: 12326.7891 - val_mae: 110.9172
Epoch 3/40
 - 6s - loss: 7767.1538 - mae: 87.3506 - val_loss: 11746.8301 - val_mae: 108.2728
Epoch 4/40
 - 6s - loss: 7128.5837 - mae: 83.6385 - val_loss: 10999.7676 - val_mae: 104.7679
Epoch 5/40
 - 6s - loss: 6372.1409 - mae: 79.0006 - val_loss: 10054.3037 - val_mae: 100.1556
Epoch 6/40
 - 6s - loss: 5441.5063 - mae: 72.7843 - val_loss: 8880.6445 - val_mae: 94.1142
Epoch 7/40
 - 6s - loss: 4300.6770 - mae: 64.5524 - val_loss: 7474.1729 - val_mae: 86.3178
Epoch 8/40
 - 6s - loss: 3046.3054 - mae: 53.9019 - val_loss: 5858.7153 - val_mae: 76.3851
Epoch 9/40
 - 6s - loss: 1768.0410 - mae: 40.4030 - val_loss: 4130.2578 - val_mae: 64.0715
Epoch 10/40
 - 6s - loss: 710.2482 - mae: 24.3777 - val_loss: 2486.1279 - val_mae: 49.5934
Epoch 11/40
 - 6s - loss: 184.2529 

In [14]:
x_test = train_x_set[226:,:]
y_test = train_y_set[226:,:]
predict_test = model.predict(x_test)

In [15]:
# predict_T = sc.inverse_transform(predict_test)
predict_test

array([[82.55432 ],
       [83.369865],
       [82.17338 ],
       [80.626015],
       [80.17622 ],
       [80.95608 ],
       [79.68409 ],
       [78.960594],
       [79.213745],
       [78.83868 ],
       [79.89079 ],
       [81.04729 ],
       [82.42825 ],
       [80.97788 ],
       [82.2717  ],
       [83.16136 ],
       [83.55228 ],
       [81.89528 ],
       [83.07763 ],
       [84.21307 ],
       [83.10919 ],
       [84.630356],
       [84.601   ],
       [83.393394],
       [81.93188 ],
       [81.45426 ],
       [82.722336],
       [82.32737 ],
       [81.85903 ],
       [83.05671 ],
       [82.53421 ],
       [83.72135 ],
       [83.98203 ],
       [83.10084 ],
       [83.35487 ],
       [85.14559 ],
       [84.30894 ],
       [85.3846  ],
       [83.46389 ],
       [84.915115],
       [83.75753 ],
       [84.54656 ],
       [84.754845],
       [83.478004],
       [83.885864],
       [85.23627 ],
       [83.09634 ],
       [85.65729 ],
       [85.25837 ],
       [85.66247 ],


In [ ]:
datay.iloc[226:,:]